# 금융 데이터 탐색적 데이터 분석 (EDA)

이 노트북은 KB 금융 지식 그래프를 위한 데이터 파일들을 탐색합니다.

## 데이터 파일 구조
- 파일 구분자: `^`
- 파일명에서 앞의 'XX_' prefix를 제외하면 테이블명
- 각 테이블별 스키마는 eda.md 파일 참조

In [68]:
import pandas as pd
import os
import glob
from pathlib import Path

# 데이터 디렉토리 경로 설정
data_dir = Path('../data')
print(f"데이터 디렉토리: {data_dir.resolve()}")

데이터 디렉토리: /Users/toby/prog/kt/202512_KB/kt-fin-kg/data


In [69]:
# 데이터 파일 목록 확인
data_files = sorted(list(data_dir.glob('*.dat')))
print(f"총 {len(data_files)}개의 데이터 파일 발견:")
for file in data_files:
    size_mb = file.stat().st_size / (1024 * 1024)
    print(f"  {file.name} ({size_mb:.2f} MB)")

총 12개의 데이터 파일 발견:
  01_CORP_PROF.dat (22.34 MB)
  02_CUST_MST.dat (23.34 MB)
  03_CORP_BIZ_MAP.dat (9.58 MB)
  04_LOAN_CTRT.dat (2.49 MB)
  05_PRD_MST.dat (0.09 MB)
  06_COLL_SET.dat (2.94 MB)
  07_CAI_TXN.dat (7.58 MB)
  08_KSIC_CD.dat (0.16 MB)
  09_ADM_DONG_CD.dat (1.48 MB)
  10_ZIP_CD.dat (1.89 MB)
  11_BRANCH_CD.dat (0.04 MB)
  12_CMN_CD.dat (0.02 MB)


In [70]:
# 각 파일별로 3개 샘플 데이터 출력하는 함수
def detect_encoding(file_path):
    """
    파일의 인코딩을 자동으로 감지하는 함수
    """
    import chardet
    
    try:
        with open(file_path, 'rb') as f:
            raw_data = f.read(10000)  # 첫 10KB만 읽어서 인코딩 감지
            result = chardet.detect(raw_data)
            return result['encoding']
    except:
        return 'utf-8'

def display_file_samples(file_path, n_samples=3):
    """
    데이터 파일에서 샘플 데이터를 읽어서 출력
    
    Args:
        file_path: 데이터 파일 경로
        n_samples: 출력할 샘플 개수
    """
    try:
        # 파일명에서 테이블명 추출 (앞의 숫자_를 제거)
        table_name = file_path.name.split('_', 1)[1].replace('.dat', '')
        
        print(f"\n{'='*60}")
        print(f"테이블: {table_name}")
        print(f"파일: {file_path.name}")
        print(f"{'='*60}")
        
        # 파일 크기 정보
        size_mb = file_path.stat().st_size / (1024 * 1024)
        print(f"파일 크기: {size_mb:.2f} MB")
        
        # 인코딩 감지 및 설정
        encodings_to_try = ['cp949', 'euc-kr', 'utf-8', 'latin-1']
        encoding_used = None
        first_line = None
        
        for encoding in encodings_to_try:
            try:
                with open(file_path, 'r', encoding=encoding) as f:
                    first_line = f.readline().strip()
                    encoding_used = encoding
                    break
            except UnicodeDecodeError:
                continue
        
        if first_line is None:
            print("파일 인코딩을 감지할 수 없습니다.")
            return
            
        print(f"사용된 인코딩: {encoding_used}")
        
        # 컬럼 수 확인
        column_count = len(first_line.split('^'))
        print(f"컬럼 수: {column_count}")
        
        # 데이터 읽기 (구분자 ^, 헤더 없음)
        df = pd.read_csv(file_path, sep='^', header=None, nrows=n_samples, encoding=encoding_used)
        
        print(f"\n샘플 데이터 ({n_samples}행):")
        print("-" * 60)
        
        # 데이터 출력 (각 행을 세로로 표시)
        for idx, row in df.iterrows():
            print(f"\n[샘플 {idx + 1}]")
            for col_idx, value in enumerate(row):
                print(f"  컬럼 {col_idx + 1}: {value}")
                
    except Exception as e:
        print(f"오류 발생: {e}")
        print(f"파일을 읽는 중 문제가 발생했습니다: {file_path.name}")

In [71]:
# 모든 데이터 파일의 샘플 데이터 출력
print("KB 금융 데이터 샘플 분석")
print("=" * 80)

for file_path in data_files:
    display_file_samples(file_path, n_samples=3)

KB 금융 데이터 샘플 분석

테이블: CORP_PROF
파일: 01_CORP_PROF.dat
파일 크기: 22.34 MB
사용된 인코딩: cp949
컬럼 수: 7

샘플 데이터 (3행):
------------------------------------------------------------

[샘플 1]
  컬럼 1: 6
  컬럼 2: 롯데**
  컬럼 3: G47111
  컬럼 4: 1048126067
  컬럼 5: 4533
  컬럼 6: 1101110000086
  컬럼 7: nan

[샘플 2]
  컬럼 1: 7
  컬럼 2: 동양**
  컬럼 3: F42311
  컬럼 4: 2018148663
  컬럼 5: 4557
  컬럼 6: 1101110000309
  컬럼 7: nan

[샘플 3]
  컬럼 1: 9
  컬럼 2: 모라*
  컬럼 3: C14300
  컬럼 4: 1098109665
  컬럼 5: 3186
  컬럼 6: 1101110000531
  컬럼 7: nan

테이블: CUST_MST
파일: 02_CUST_MST.dat
파일 크기: 23.34 MB
사용된 인코딩: cp949
컬럼 수: 5

샘플 데이터 (3행):
------------------------------------------------------------

[샘플 1]
  컬럼 1: 7000000180.0
  컬럼 2: 1018100452.0
  컬럼 3: 2.0
  컬럼 4: 4.0
  컬럼 5: nan

[샘플 2]
  컬럼 1: 7000000233.0
  컬럼 2: 1018110436.0
  컬럼 3: 2.0
  컬럼 4: 4.0
  컬럼 5: nan

[샘플 3]
  컬럼 1: 7000000291.0
  컬럼 2: 1018118476.0
  컬럼 3: 2.0
  컬럼 4: 4.0
  컬럼 5: nan

테이블: CORP_BIZ_MAP
파일: 03_CORP_BIZ_MAP.dat
파일 크기: 9.58 MB
사용된 인코딩: cp949
컬럼 수: 3

샘플 데이터 (3

In [72]:
# 전체 데이터 파일 요약 정보
print("\n" + "=" * 80)
print("데이터 파일 요약")
print("=" * 80)

total_size = 0
file_info = []

for file_path in data_files:
    size_mb = file_path.stat().st_size / (1024 * 1024)
    total_size += size_mb
    
    # 행 수 계산 (빠른 계산을 위해 wc 명령 사용)
    try:
        import subprocess
        result = subprocess.run(['wc', '-l', str(file_path)], 
                              capture_output=True, text=True)
        line_count = int(result.stdout.split()[0])
    except:
        line_count = "N/A"
    
    # 인코딩 확인
    encodings_to_try = ['cp949', 'euc-kr', 'utf-8', 'latin-1']
    encoding_used = 'unknown'
    
    for encoding in encodings_to_try:
        try:
            with open(file_path, 'r', encoding=encoding) as f:
                f.readline()  # 첫 줄만 읽어서 테스트
                encoding_used = encoding
                break
        except UnicodeDecodeError:
            continue
    
    table_name = file_path.name.split('_', 1)[1].replace('.dat', '')
    file_info.append({
        '테이블명': table_name,
        '파일명': file_path.name,
        '크기(MB)': f"{size_mb:.2f}",
        '행수': line_count,
        '인코딩': encoding_used
    })

# 요약 테이블 출력
summary_df = pd.DataFrame(file_info)
print(summary_df.to_string(index=False))
print(f"\n총 파일 크기: {total_size:.2f} MB")


데이터 파일 요약


        테이블명                 파일명 크기(MB)     행수   인코딩
   CORP_PROF    01_CORP_PROF.dat  22.34 395108 cp949
    CUST_MST     02_CUST_MST.dat  23.34 831796 cp949
CORP_BIZ_MAP 03_CORP_BIZ_MAP.dat   9.58 436688 cp949
   LOAN_CTRT    04_LOAN_CTRT.dat   2.49  25203 cp949
     PRD_MST      05_PRD_MST.dat   0.09   1342 cp949
    COLL_SET     06_COLL_SET.dat   2.94  25203 cp949
     CAI_TXN      07_CAI_TXN.dat   7.58 254928 cp949
     KSIC_CD      08_KSIC_CD.dat   0.16   1205 cp949
 ADM_DONG_CD  09_ADM_DONG_CD.dat   1.48  25071 cp949
      ZIP_CD       10_ZIP_CD.dat   1.89  69675 cp949
   BRANCH_CD    11_BRANCH_CD.dat   0.04   1130 cp949
      CMN_CD       12_CMN_CD.dat   0.02    406 cp949

총 파일 크기: 71.94 MB


python(13038) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13040) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13041) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13042) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13043) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13044) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13045) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13046) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13047) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13048) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(13049) Malloc

In [73]:
# df_total 딕셔너리 생성 - 모든 데이터 파일을 로드
def create_df_total_dict(data_dir):
    """
    모든 .dat 파일을 읽어서 df_total 딕셔너리 생성
    """
    df_total = {}
    data_files = sorted(list(data_dir.glob('*.dat')))
    
    print("데이터 로딩 시작...")
    print("=" * 50)
    
    for file_path in data_files:
        try:
            # 테이블명 추출
            table_name = file_path.name.split('_', 1)[1].replace('.dat', '')
            
            # 인코딩 감지 및 데이터 로드
            encodings_to_try = ['cp949', 'euc-kr', 'utf-8', 'latin-1']
            encoding_used = None
            
            for encoding in encodings_to_try:
                try:
                    df = pd.read_csv(file_path, sep='^', header=None, encoding=encoding)
                    encoding_used = encoding
                    break
                except UnicodeDecodeError:
                    continue
            
            if df is not None:
                df_total[table_name] = df
                print(f"✓ {table_name}: {df.shape[0]:,}행 x {df.shape[1]}열 (인코딩: {encoding_used})")
            else:
                print(f"✗ {table_name}: 로드 실패")
                
        except Exception as e:
            print(f"✗ {file_path.name}: 오류 - {e}")
    
    print("=" * 50)
    print(f"총 {len(df_total)}개 테이블 로드 완료")
    return df_total

# 실제 데이터 로드 실행
df_total = create_df_total_dict(data_dir)
print(f"\n로드된 테이블: {list(df_total.keys())}")

데이터 로딩 시작...
✓ CORP_PROF: 395,108행 x 7열 (인코딩: cp949)
✓ CUST_MST: 831,796행 x 5열 (인코딩: cp949)
✓ CORP_BIZ_MAP: 436,688행 x 3열 (인코딩: cp949)
✓ LOAN_CTRT: 25,203행 x 14열 (인코딩: cp949)
✓ PRD_MST: 1,342행 x 7열 (인코딩: cp949)
✓ COLL_SET: 25,203행 x 8열 (인코딩: cp949)
✓ CAI_TXN: 254,928행 x 4열 (인코딩: cp949)
✓ KSIC_CD: 1,205행 x 11열 (인코딩: cp949)
✓ ADM_DONG_CD: 25,071행 x 12열 (인코딩: cp949)
✓ ZIP_CD: 69,675행 x 5열 (인코딩: cp949)
✓ BRANCH_CD: 1,130행 x 6열 (인코딩: cp949)
✓ CMN_CD: 406행 x 6열 (인코딩: cp949)
총 12개 테이블 로드 완료

로드된 테이블: ['CORP_PROF', 'CUST_MST', 'CORP_BIZ_MAP', 'LOAN_CTRT', 'PRD_MST', 'COLL_SET', 'CAI_TXN', 'KSIC_CD', 'ADM_DONG_CD', 'ZIP_CD', 'BRANCH_CD', 'CMN_CD']


## EDA 결과 요약 및 인사이트

위의 분석을 통해 다음과 같은 인사이트를 얻을 수 있습니다:

### 주요 발견사항
1. **데이터 품질**: 각 테이블의 결측값 비율과 중복 데이터 현황
2. **변수 유형**: Nominal vs Numerical 변수의 분포와 특성
3. **테이블 관계**: 잠재적 외래키 관계 및 테이블 간 연결점
4. **데이터 분포**: 각 변수별 분포 특성과 이상치 현황

### 추가 분석 권장사항
- 시간 시계열 분석 (날짜 컬럼이 있는 경우)
- 지리적 분석 (주소/지역 코드 기반)
- 고객 세그멘테이션 분석
- 위험 요소 분석 (대출, 담보 관련)

In [74]:
# 데이터 품질 및 관계 분석 실행
try:
    # 데이터 품질 분석
    quality_results = analyze_data_quality(df_total)
    
    # 테이블 간 관계 분석
    relationships = analyze_table_relationships(df_total)
    
except NameError:
    print("df_total 딕셔너리가 정의되지 않았습니다.")
    print("먼저 데이터를 로드해주세요.")
except Exception as e:
    print(f"분석 중 오류 발생: {e}")


################################################################################
데이터 품질 종합 분석
################################################################################
        테이블명      행수  컬럼수      총_셀수   결측_셀수 결측_비율(%) 중복_행수 중복_비율(%)
   CORP_PROF 395,108    7 2,765,756 395,108    14.29     0     0.00
    CUST_MST 831,796    5 4,158,980 831,796    20.00     0     0.00
CORP_BIZ_MAP 436,688    3 1,310,064 436,688    33.33     0     0.00
   LOAN_CTRT  25,203   14   352,842  49,079    13.91     0     0.00
     PRD_MST   1,342    7     9,394   1,342    14.29     0     0.00
    COLL_SET  25,203    8   201,624  25,203    12.50     0     0.00
     CAI_TXN 254,928    4 1,019,712 254,928    25.00     0     0.00
     KSIC_CD   1,205   11    13,255   1,205     9.09     0     0.00
 ADM_DONG_CD  25,071   12   300,852  31,702    10.54     0     0.00
      ZIP_CD  69,675    5   348,375  69,949    20.08     0     0.00
   BRANCH_CD   1,130    6     6,780   1,130    16.67     0     0.00
      CM

In [75]:
# 전체 테이블 EDA 실행
print("KB 금융 데이터 전체 EDA 분석 시작")
print("="*100)

if 'df_total' in locals() and len(df_total) > 0:
    print(f"총 테이블 수: {len(df_total)}")
    print(f"테이블 목록: {list(df_total.keys())}")
    print("\n각 테이블별 상세 분석을 시작합니다...")
    
    # 각 테이블별로 EDA 수행
    for table_name, df in df_total.items():
        perform_table_eda(df, table_name)
        
        # 메모리 절약을 위해 중간에 일시정지
        print("\n" + "-"*50 + " 다음 테이블로... " + "-"*50 + "\n")
        
else:
    print("df_total 딕셔너리가 정의되지 않았거나 비어있습니다.")
    print("먼저 앞의 데이터 로딩 셀을 실행해주세요.")

KB 금융 데이터 전체 EDA 분석 시작
총 테이블 수: 12
테이블 목록: ['CORP_PROF', 'CUST_MST', 'CORP_BIZ_MAP', 'LOAN_CTRT', 'PRD_MST', 'COLL_SET', 'CAI_TXN', 'KSIC_CD', 'ADM_DONG_CD', 'ZIP_CD', 'BRANCH_CD', 'CMN_CD']

각 테이블별 상세 분석을 시작합니다...

################################################################################
테이블: CORP_PROF
################################################################################
데이터 형태: (395108, 7)
컬럼 수: 7
행 수: 395,108

수치형 변수 분석: 컬럼 1
총 데이터 개수: 395,108
결측값 개수: 0
결측값 비율: 0.00%

기술통계:
------------------------------
평균: 6,366,884.44
중앙값: 7,722,932.50
표준편차: 22,069,986.75
최솟값: 6.00
최댓값: 9,014,000,001.00
25% 분위수: 5,143,452.75
50% 분위수: 7,722,932.50
75% 분위수: 8,245,649.25

이상치 분석:
------------------------------
이상치 개수: 42,095
이상치 비율: 10.65%
이상치 범위: 490158.00 미만 또는 12898944.00 초과

범주형 변수 분석: 컬럼 2
총 데이터 개수: 395,108
고유값 개수: 87,586
결측값 개수: 0
결측값 비율: 0.00%

상위 20개 카테고리:
------------------------------
 1. 에스****                         :    1,792 ( 0.45%)
 2. 에스***                        

In [76]:
# 데이터 품질 및 관계 분석 실행
if 'df_total' in locals() and len(df_total) > 0:
    print("데이터 품질 및 관계 분석 시작...")
    
    # 데이터 품질 분석
    quality_results = analyze_data_quality(df_total)
    
    # 테이블 간 관계 분석 
    print("\\n테이블 간 관계 분석을 시작합니다 (시간이 걸릴 수 있습니다)...")
    relationships = analyze_table_relationships(df_total)
    
    print("\\n" + "="*100)
    print("전체 EDA 분석이 완료되었습니다!")
    print("="*100)
    
else:
    print("df_total 딕셔너리가 정의되지 않았습니다.")
    print("먼저 데이터 로딩 셀을 실행해주세요.")

데이터 품질 및 관계 분석 시작...

################################################################################
데이터 품질 종합 분석
################################################################################
        테이블명      행수  컬럼수      총_셀수   결측_셀수 결측_비율(%) 중복_행수 중복_비율(%)
   CORP_PROF 395,108    7 2,765,756 395,108    14.29     0     0.00
    CUST_MST 831,796    5 4,158,980 831,796    20.00     0     0.00
CORP_BIZ_MAP 436,688    3 1,310,064 436,688    33.33     0     0.00
   LOAN_CTRT  25,203   14   352,842  49,079    13.91     0     0.00
     PRD_MST   1,342    7     9,394   1,342    14.29     0     0.00
    COLL_SET  25,203    8   201,624  25,203    12.50     0     0.00
     CAI_TXN 254,928    4 1,019,712 254,928    25.00     0     0.00
     KSIC_CD   1,205   11    13,255   1,205     9.09     0     0.00
 ADM_DONG_CD  25,071   12   300,852  31,702    10.54     0     0.00
      ZIP_CD  69,675    5   348,375  69,949    20.08     0     0.00
   BRANCH_CD   1,130    6     6,780   1,130    16.67   

In [77]:
# 개별 테이블 선택적 분석 (필요시 사용)
def analyze_specific_table(table_name, df_total):
    """
    특정 테이블만 선택해서 분석
    """
    if table_name in df_total:
        print(f"{table_name} 테이블 개별 분석")
        perform_table_eda(df_total[table_name], table_name)
    else:
        print(f"테이블 '{table_name}'을 찾을 수 없습니다.")
        print(f"사용 가능한 테이블: {list(df_total.keys())}")

# 사용 예시 (필요한 테이블명으로 변경)
# analyze_specific_table('CORP_PROF', df_total)
# analyze_specific_table('LOAN_CTRT', df_total)

print("개별 테이블 분석을 원하시면 위의 주석을 해제하고 테이블명을 지정해서 실행하세요.")
print(f"사용 가능한 테이블: {list(df_total.keys()) if 'df_total' in locals() else '데이터를 먼저 로드하세요'}")

개별 테이블 분석을 원하시면 위의 주석을 해제하고 테이블명을 지정해서 실행하세요.
사용 가능한 테이블: ['CORP_PROF', 'CUST_MST', 'CORP_BIZ_MAP', 'LOAN_CTRT', 'PRD_MST', 'COLL_SET', 'CAI_TXN', 'KSIC_CD', 'ADM_DONG_CD', 'ZIP_CD', 'BRANCH_CD', 'CMN_CD']


In [78]:
# EDA 분석 함수 정의
def analyze_nominal_variable(series, var_name, max_categories=20):
    """
    Nominal (범주형) 변수 분석
    """
    print(f"\n{'='*50}")
    print(f"범주형 변수 분석: {var_name}")
    print(f"{'='*50}")
    
    # 기본 정보
    print(f"총 데이터 개수: {len(series):,}")
    print(f"고유값 개수: {series.nunique():,}")
    print(f"결측값 개수: {series.isnull().sum():,}")
    print(f"결측값 비율: {series.isnull().mean()*100:.2f}%")
    
    # 빈도 분석
    value_counts = series.value_counts()
    print(f"\n상위 {min(max_categories, len(value_counts))}개 카테고리:")
    print("-" * 30)
    
    for i, (value, count) in enumerate(value_counts.head(max_categories).items()):
        percentage = (count / len(series)) * 100
        print(f"{i+1:2d}. {str(value)[:30]:30s} : {count:8,} ({percentage:5.2f}%)")
    
    # 분포 특성
    if len(value_counts) > max_categories:
        print(f"\n... (총 {len(value_counts)}개 카테고리 중 상위 {max_categories}개만 표시)")
    
    return value_counts

def analyze_numerical_variable(series, var_name):
    """
    Numerical (수치형) 변수 분석
    """
    print(f"\n{'='*50}")
    print(f"수치형 변수 분석: {var_name}")
    print(f"{'='*50}")
    
    # 기본 정보
    print(f"총 데이터 개수: {len(series):,}")
    print(f"결측값 개수: {series.isnull().sum():,}")
    print(f"결측값 비율: {series.isnull().mean()*100:.2f}%")
    
    # 수치형으로 변환 가능한지 확인
    try:
        numeric_series = pd.to_numeric(series, errors='coerce')
        
        if numeric_series.isnull().all():
            print("수치형으로 변환할 수 없는 데이터입니다.")
            return analyze_nominal_variable(series, var_name)
        
        # 기술통계
        print(f"\n기술통계:")
        print("-" * 30)
        print(f"평균: {numeric_series.mean():,.2f}")
        print(f"중앙값: {numeric_series.median():,.2f}")
        print(f"표준편차: {numeric_series.std():,.2f}")
        print(f"최솟값: {numeric_series.min():,.2f}")
        print(f"최댓값: {numeric_series.max():,.2f}")
        
        # 분위수
        quantiles = numeric_series.quantile([0.25, 0.5, 0.75])
        print(f"25% 분위수: {quantiles[0.25]:,.2f}")
        print(f"50% 분위수: {quantiles[0.5]:,.2f}")
        print(f"75% 분위수: {quantiles[0.75]:,.2f}")
        
        # 이상치 탐지 (IQR 방법)
        Q1 = quantiles[0.25]
        Q3 = quantiles[0.75]
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        outliers = numeric_series[(numeric_series < lower_bound) | (numeric_series > upper_bound)]
        print(f"\n이상치 분석:")
        print("-" * 30)
        print(f"이상치 개수: {len(outliers):,}")
        print(f"이상치 비율: {len(outliers)/len(numeric_series)*100:.2f}%")
        
        if len(outliers) > 0:
            print(f"이상치 범위: {lower_bound:.2f} 미만 또는 {upper_bound:.2f} 초과")
        
        return numeric_series
        
    except Exception as e:
        print(f"수치형 분석 중 오류: {e}")
        return analyze_nominal_variable(series, var_name)

def detect_variable_type(series, var_name):
    """
    변수 타입을 자동으로 감지 (nominal vs numerical)
    """
    # 고유값 비율로 판단
    unique_ratio = series.nunique() / len(series)
    
    # 수치형으로 변환 시도
    try:
        numeric_series = pd.to_numeric(series, errors='coerce')
        numeric_ratio = (~numeric_series.isnull()).mean()
        
        # 판단 기준
        if numeric_ratio > 0.8 and unique_ratio > 0.1:  # 80% 이상이 숫자이고, 고유값 비율이 10% 이상
            return 'numerical'
        else:
            return 'nominal'
    except:
        return 'nominal'

In [79]:
# EDA 분석을 위한 추가 라이브러리 import
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False

# 시각화 스타일 설정
sns.set_style("whitegrid")
plt.style.use('default')

In [80]:
# 각 테이블별 전체 EDA 실행 함수
def perform_table_eda(df, table_name):
    """
    테이블별 종합 EDA 수행
    """
    print(f"\n{'#'*80}")
    print(f"테이블: {table_name}")
    print(f"{'#'*80}")
    
    print(f"데이터 형태: {df.shape}")
    print(f"컬럼 수: {df.shape[1]}")
    print(f"행 수: {df.shape[0]:,}")
    
    # 각 컬럼별 분석
    for col_idx, col in enumerate(df.columns):
        series = df[col].dropna()  # 결측값 제거 후 분석
        
        if len(series) == 0:
            print(f"\n컬럼 {col_idx + 1}: 모든 값이 결측값입니다.")
            continue
        
        # 변수 타입 자동 감지
        var_type = detect_variable_type(series, f"컬럼 {col_idx + 1}")
        
        if var_type == 'numerical':
            analyze_numerical_variable(series, f"컬럼 {col_idx + 1}")
        else:
            analyze_nominal_variable(series, f"컬럼 {col_idx + 1}")
    
    print(f"\n{'='*80}")
    print(f"{table_name} 테이블 분석 완료")
    print(f"{'='*80}\n")

# 데이터 품질 분석 함수
def analyze_data_quality(df_total):
    """
    전체 데이터 품질 분석
    """
    print(f"\n{'#'*80}")
    print("데이터 품질 종합 분석")
    print(f"{'#'*80}")
    
    quality_summary = []
    
    for table_name, df in df_total.items():
        # 기본 정보
        total_cells = df.shape[0] * df.shape[1]
        missing_cells = df.isnull().sum().sum()
        missing_percentage = (missing_cells / total_cells) * 100
        
        # 중복 행 확인
        duplicate_rows = df.duplicated().sum()
        duplicate_percentage = (duplicate_rows / len(df)) * 100
        
        quality_summary.append({
            '테이블명': table_name,
            '행수': f"{df.shape[0]:,}",
            '컬럼수': df.shape[1],
            '총_셀수': f"{total_cells:,}",
            '결측_셀수': f"{missing_cells:,}",
            '결측_비율(%)': f"{missing_percentage:.2f}",
            '중복_행수': f"{duplicate_rows:,}",
            '중복_비율(%)': f"{duplicate_percentage:.2f}"
        })
    
    # 품질 요약 테이블 출력
    quality_df = pd.DataFrame(quality_summary)
    print(quality_df.to_string(index=False))
    
    return quality_df

# 테이블 간 관계 분석 함수
def analyze_table_relationships(df_total):
    """
    테이블 간 관계 분석 (공통 컬럼 및 값 기반)
    """
    print(f"\n{'#'*80}")
    print("테이블 간 관계 분석")
    print(f"{'#'*80}")
    
    tables = list(df_total.keys())
    relationships = []
    
    # 각 테이블 쌍에 대해 공통 값 분석
    for i, table1 in enumerate(tables):
        for j, table2 in enumerate(tables[i+1:], i+1):
            df1, df2 = df_total[table1], df_total[table2]
            
            # 각 컬럼 조합에서 공통 값 찾기
            for col1_idx in range(df1.shape[1]):
                for col2_idx in range(df2.shape[1]):
                    try:
                        col1_values = set(df1.iloc[:, col1_idx].dropna().astype(str))
                        col2_values = set(df2.iloc[:, col2_idx].dropna().astype(str))
                        
                        common_values = col1_values.intersection(col2_values)
                        
                        if len(common_values) > 0:
                            overlap_ratio = len(common_values) / min(len(col1_values), len(col2_values))
                            
                            # 의미있는 관계로 판단되는 기준
                            if overlap_ratio > 0.1 and len(common_values) > 10:
                                relationships.append({
                                    '테이블1': table1,
                                    '컬럼1': f"컬럼{col1_idx+1}",
                                    '테이블2': table2,
                                    '컬럼2': f"컬럼{col2_idx+1}",
                                    '공통값수': len(common_values),
                                    '겹침비율': f"{overlap_ratio:.3f}"
                                })
                    except Exception as e:
                        continue
    
    if relationships:
        rel_df = pd.DataFrame(relationships)
        rel_df = rel_df.sort_values('겹침비율', ascending=False)
        print("잠재적 테이블 관계:")
        print(rel_df.to_string(index=False))
    else:
        print("명확한 테이블 간 관계를 찾을 수 없습니다.")
    
    return relationships

# 상세 탐색적 데이터 분석 (EDA)

이제 각 테이블별로 상세한 EDA를 진행합니다.
- **Nominal 변수**: 범주형 데이터의 빈도 분석, 고유값 확인
- **Numerical 변수**: 기술통계, 분포 분석, 이상치 탐지

In [ ]:
# 6단계: EDA 결과 종합 분석 및 인사이트
print("\n" + "=" * 100)
print("6단계: EDA 결과 종합 분석 및 주요 인사이트")
print("=" * 100)

# 수치형 변수들의 요약 통계
numerical_summary = []

if 'df_total' in locals():
    print(f"\n{'='*60}")
    print("주요 수치형 변수 요약")
    print(f"{'='*60}")
    
    # LOAN_CTRT 대줄잔액 요약
    if 'LOAN_CTRT' in df_total:
        loan_amounts = pd.to_numeric(df_total['LOAN_CTRT'].iloc[:, 9], errors='coerce')
        numerical_summary.append({
            '변수명': '대줄잔액',
            '테이블': 'LOAN_CTRT',
            '평균': f"{loan_amounts.mean():,.0f}원",
            '중앙값': f"{loan_amounts.median():,.0f}원",
            '최댓값': f"{loan_amounts.max():,.0f}원",
            '표준편차': f"{loan_amounts.std():,.0f}원"
        })
    
    # COLL_SET 담보평가금액 요약
    if 'COLL_SET' in df_total:
        coll_amounts = pd.to_numeric(df_total['COLL_SET'].iloc[:, 2], errors='coerce')
        numerical_summary.append({
            '변수명': '담보평가금액',
            '테이블': 'COLL_SET', 
            '평균': f"{coll_amounts.mean():,.0f}원",
            '중앙값': f"{coll_amounts.median():,.0f}원",
            '최댓값': f"{coll_amounts.max():,.0f}원",
            '표준편차': f"{coll_amounts.std():,.0f}원"
        })
        
        # LTV 분석
        ltv_values = pd.to_numeric(df_total['COLL_SET'].iloc[:, 3], errors='coerce')
        numerical_summary.append({
            '변수명': 'LTV비율',
            '테이블': 'COLL_SET',
            '평균': f"{ltv_values.mean():.2f}%",
            '중앙값': f"{ltv_values.median():.2f}%", 
            '최댓값': f"{ltv_values.max():.2f}%",
            '표준편차': f"{ltv_values.std():.2f}%"
        })
    
    # CAI_TXN 매출거래금액 요약
    if 'CAI_TXN' in df_total:
        txn_amounts = pd.to_numeric(df_total['CAI_TXN'].iloc[:, 2], errors='coerce')
        numerical_summary.append({
            '변수명': '매출거래금액',
            '테이블': 'CAI_TXN',
            '평균': f"{txn_amounts.mean():,.0f}원",
            '중앙값': f"{txn_amounts.median():,.0f}원", 
            '최댓값': f"{txn_amounts.max():,.0f}원",
            '표준편차': f"{txn_amounts.std():,.0f}원"
        })
    
    # 요약 테이블 출력
    if numerical_summary:
        summary_df = pd.DataFrame(numerical_summary)
        print(summary_df.to_string(index=False))
    
    # 주요 인사이트
    print(f"\n{'='*60}")
    print("주요 발견 인사이트")
    print(f"{'='*60}")
    
    insights = [
        "🏦 대출 포트폴리오 분석:",
        f"   - 총 대출 계약 수: {df_total['LOAN_CTRT'].shape[0]:,}건" if 'LOAN_CTRT' in df_total else "",
        f"   - 총 담보 설정 수: {df_total['COLL_SET'].shape[0]:,}건" if 'COLL_SET' in df_total else "",
        "",
        "🏢 기업 데이터 현황:",
        f"   - 등록 기업 수: {df_total['CORP_PROF'].shape[0]:,}개" if 'CORP_PROF' in df_total else "",
        f"   - B2B 거래 관계: {df_total['CAI_TXN'].shape[0]:,}건" if 'CAI_TXN' in df_total else "",
        "",
        "📊 데이터 품질 특징:",
        "   - 대부분 테이블이 cp949 인코딩",
        "   - 마지막 컬럼에 결측값 집중",
        "   - 수치형/범주형 변수 혼재"
    ]
    
    for insight in insights:
        if insight:  # 빈 문자열이 아닌 경우만 출력
            print(insight)

print(f"\n{'='*100}")
print("상세 EDA 분석 완료!")
print(f"{'='*100}")

In [ ]:
# 5단계: 코드 테이블들 상세 분석
print("\n" + "=" * 100)
print("5단계: 코드 테이블들 (KSIC_CD, ADM_DONG_CD, ZIP_CD) 상세 EDA")
print("=" * 100)

# KSIC_CD (표준산업분류코드) 분석
if 'df_total' in locals() and 'KSIC_CD' in df_total:
    ksic_cd = df_total['KSIC_CD']
    
    print(f"\n{'='*60}")
    print("KSIC_CD (표준산업분류코드) 분석")
    print(f"{'='*60}")
    print(f"테이블 크기: {ksic_cd.shape[0]:,}행 x {ksic_cd.shape[1]}열")
    
    # 대분류코드
    print("\n[컬럼 9: 대분류코드]")
    analyze_nominal_variable(ksic_cd.iloc[:, 8], "대분류코드", max_categories=25)
    
    # 대분류명
    print("\n[컬럼 10: 대분류명]")
    analyze_nominal_variable(ksic_cd.iloc[:, 9], "대분류명", max_categories=25)

# ADM_DONG_CD (법정동행정구역) 분석  
if 'df_total' in locals() and 'ADM_DONG_CD' in df_total:
    adm_dong = df_total['ADM_DONG_CD']
    
    print(f"\n{'='*60}")
    print("ADM_DONG_CD (법정동행정구역) 분석")
    print(f"{'='*60}")
    print(f"테이블 크기: {adm_dong.shape[0]:,}행 x {adm_dong.shape[1]}열")
    
    # 광역시도명
    print("\n[컬럼 8: 광역시도명]")
    analyze_nominal_variable(adm_dong.iloc[:, 7], "광역시도명", max_categories=20)
    
    # 시군구명
    print("\n[컬럼 9: 시군구명]") 
    analyze_nominal_variable(adm_dong.iloc[:, 8], "시군구명", max_categories=20)

# ZIP_CD (우편번호) 분석
if 'df_total' in locals() and 'ZIP_CD' in df_total:
    zip_cd = df_total['ZIP_CD']
    
    print(f"\n{'='*60}")
    print("ZIP_CD (우편번호) 분석")
    print(f"{'='*60}")
    print(f"테이블 크기: {zip_cd.shape[0]:,}행 x {zip_cd.shape[1]}열")
    
    # 우편번호
    print("\n[컬럼 1: 우편번호]")
    analyze_numerical_variable(zip_cd.iloc[:, 0], "우편번호")
    
    # 한글시도명
    print("\n[컬럼 2: 한글시도명]")
    analyze_nominal_variable(zip_cd.iloc[:, 1], "한글시도명", max_categories=20)

In [ ]:
# 4단계: CAI_TXN 테이블 상세 분석 (기업간거래관계)
print("\n" + "=" * 100)
print("4단계: CAI_TXN (기업간거래관계분석) 테이블 상세 EDA")
print("=" * 100)

if 'df_total' in locals() and 'CAI_TXN' in df_total:
    cai_txn = df_total['CAI_TXN']
    
    print(f"테이블 크기: {cai_txn.shape[0]:,}행 x {cai_txn.shape[1]}열")
    print(f"메모리 사용량: {cai_txn.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    print(f"\n{'='*60}")
    print("CAI_TXN 기업간 거래 분석")
    print(f"{'='*60}")
    
    # 판매사업자번호
    print("\n[컬럼 1: 판매사업자번호]")
    analyze_nominal_variable(cai_txn.iloc[:, 0], "판매사업자번호", max_categories=15)
    
    # 구매사업자번호  
    print("\n[컬럼 2: 구매사업자번호]")
    analyze_nominal_variable(cai_txn.iloc[:, 1], "구매사업자번호", max_categories=15)
    
    # 매출거래금액 (핵심 수치 변수)
    print("\n[컬럼 3: 매출거래금액]")
    analyze_numerical_variable(cai_txn.iloc[:, 2], "매출거래금액")
    
else:
    print("CAI_TXN 테이블을 찾을 수 없습니다.")

In [ ]:
# 3단계: COLL_SET 테이블 상세 분석 (담보 데이터)
print("\n" + "=" * 100)
print("3단계: COLL_SET (담보설정) 테이블 상세 EDA")
print("=" * 100)

if 'df_total' in locals() and 'COLL_SET' in df_total:
    coll_set = df_total['COLL_SET']
    
    print(f"테이블 크기: {coll_set.shape[0]:,}행 x {coll_set.shape[1]}열")
    print(f"메모리 사용량: {coll_set.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # 담보 관련 핵심 지표 분석
    print(f"\n{'='*60}")
    print("COLL_SET 담보 관련 핵심 지표 분석")
    print(f"{'='*60}")
    
    # 담보여신계좌번호
    print("\n[컬럼 1: 담보여신계좌번호]")
    analyze_nominal_variable(coll_set.iloc[:, 0], "담보여신계좌번호", max_categories=10)
    
    # 대출잔액
    print("\n[컬럼 2: 대출잔액]")
    analyze_numerical_variable(coll_set.iloc[:, 1], "대출잔액")
    
    # 담보평가금액
    print("\n[컬럼 3: 담보평가금액]")
    analyze_numerical_variable(coll_set.iloc[:, 2], "담보평가금액")
    
    # 담보대대출비율 (LTV - 중요한 리스크 지표)
    print("\n[컬럼 4: 담보대대출비율(LTV)]")
    analyze_numerical_variable(coll_set.iloc[:, 3], "담보대대출비율(LTV)")
    
    # 법정동코드 (지역 분석)
    print("\n[컬럼 6: 법정동코드]")
    analyze_nominal_variable(coll_set.iloc[:, 5], "법정동코드", max_categories=15)
    
    # 주담보구분
    print("\n[컬럼 7: 주담보구분]")
    analyze_nominal_variable(coll_set.iloc[:, 6], "주담보구분", max_categories=10)
    
else:
    print("COLL_SET 테이블을 찾을 수 없습니다.")

In [ ]:
# 2단계: LOAN_CTRT 테이블 상세 분석
print("\n" + "=" * 100)
print("2단계: LOAN_CTRT (여신계약) 테이블 상세 EDA")
print("=" * 100)

if 'df_total' in locals() and 'LOAN_CTRT' in df_total:
    loan_ctrt = df_total['LOAN_CTRT']
    
    print(f"테이블 크기: {loan_ctrt.shape[0]:,}행 x {loan_ctrt.shape[1]}열")
    print(f"메모리 사용량: {loan_ctrt.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # 주요 컬럼 분석
    print(f"\n{'='*60}")
    print("LOAN_CTRT 주요 컬럼 분석")
    print(f"{'='*60}")
    
    # 계좌번호
    print("\n[컬럼 1: 계좌번호]")
    analyze_nominal_variable(loan_ctrt.iloc[:, 0], "계좌번호", max_categories=10)
    
    # 계약자고객식별자
    print("\n[컬럼 3: 계약자고객식별자]")
    analyze_nominal_variable(loan_ctrt.iloc[:, 2], "계약자고객식별자", max_categories=10)
    
    # 상품코드
    print("\n[컬럼 5: 상품코드]")
    analyze_nominal_variable(loan_ctrt.iloc[:, 4], "상품코드", max_categories=15)
    
    # 대줄잔액 (핵심 수치 변수)
    print("\n[컬럼 10: 대줄잔액]")
    analyze_numerical_variable(loan_ctrt.iloc[:, 9], "대줄잔액")
    
    # 관리부점코드
    print("\n[컬럼 8: 관리부점코드]")
    analyze_nominal_variable(loan_ctrt.iloc[:, 7], "관리부점코드", max_categories=15)
    
    # 통화코드
    print("\n[컬럼 9: 통화코드]")
    analyze_nominal_variable(loan_ctrt.iloc[:, 8], "통화코드", max_categories=10)
    
else:
    print("LOAN_CTRT 테이블을 찾을 수 없습니다.")

In [ ]:
# 1단계: CORP_PROF 테이블 상세 분석
print("=" * 100)
print("1단계: CORP_PROF (기업개황) 테이블 상세 EDA")
print("=" * 100)

if 'df_total' in locals() and 'CORP_PROF' in df_total:
    corp_prof = df_total['CORP_PROF']
    
    print(f"테이블 크기: {corp_prof.shape[0]:,}행 x {corp_prof.shape[1]}열")
    print(f"메모리 사용량: {corp_prof.memory_usage(deep=True).sum() / (1024**2):.2f} MB")
    
    # 각 컬럼별 상세 분석
    print(f"\n{'='*60}")
    print("CORP_PROF 컬럼별 분석")
    print(f"{'='*60}")
    
    # 컬럼 1: 기업식별자
    print("\n[컬럼 1: 기업식별자]")
    analyze_numerical_variable(corp_prof.iloc[:, 0], "기업식별자")
    
    # 컬럼 2: 크레탑업체명  
    print("\n[컬럼 2: 크레탑업체명]")
    analyze_nominal_variable(corp_prof.iloc[:, 1], "크레탑업체명", max_categories=15)
    
    # 컬럼 3: 크레탑업종구분
    print("\n[컬럼 3: 크레탑업종구분]") 
    analyze_nominal_variable(corp_prof.iloc[:, 2], "크레탑업종구분", max_categories=20)
    
    # 컬럼 4: 사업자번호
    print("\n[컬럼 4: 사업자번호]")
    analyze_numerical_variable(corp_prof.iloc[:, 3], "사업자번호")
    
    # 컬럼 5: 소재지우편번호
    print("\n[컬럼 5: 소재지우편번호]")
    analyze_nominal_variable(corp_prof.iloc[:, 4], "소재지우편번호", max_categories=15)
    
    # 컬럼 6: 법인등록번호
    print("\n[컬럼 6: 법인등록번호]")
    analyze_numerical_variable(corp_prof.iloc[:, 5], "법인등록번호")
    
else:
    print("CORP_PROF 테이블을 찾을 수 없습니다.")